In [1]:
import sys
sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append("/home/webphy/Desktop/dnn_processor/dataset/")  # just to enable `dataset`

import tensorflow as tf
import dataset
import numpy as np
import keras
from datetime import datetime
from keras import layers

2024-02-10 16:52:27.103173: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-10 16:52:27.126591: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 16:52:27.126615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 16:52:27.127336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 16:52:27.131832: I tensorflow/core/platform/cpu_feature_guar

In [2]:
DEFAULT_BATCH_SIZE = 128
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set_4x/*/*.jpeg"
test_images_rgx = "../../dataset/test_set_4x/*/*.jpeg"

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

2024-02-10 16:52:28.042013: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 16:52:28.064896: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 16:52:28.065031: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def create_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="accuracy",
        min_delta=0.01,
        patience=21,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )


def create_lr_reducer_on_plateau():
    return keras.callbacks.ReduceLROnPlateau(
        monitor="accuracy", factor=0.5, patience=10, mode="max", verbose=1, start_from_epoch=50
    )

In [4]:
def create_augmentation_layers(x):

    x = layers.RandomBrightness(0.1, (0, 1.0))(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomTranslation(0.15, 0.15)(x)
    x = layers.GaussianNoise(0.1)(x)

    return x

In [5]:
x = x_in = layers.Input(shape=(DEFAULT_INPUT_SIZE, DEFAULT_INPUT_SIZE, 3))

x = create_augmentation_layers(x)

width_multiplier = 10

x = keras.layers.Conv2D(4 * width_multiplier, 4, 4)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Conv2D(16 * width_multiplier, 3, 2)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Conv2D(5 * width_multiplier, 1, 1)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(num_classes)(x)

# Build model
model = keras.Model(x_in, x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 random_brightness (RandomB  (None, 128, 128, 3)       0         
 rightness)                                                      
                                                                 
 random_flip (RandomFlip)    (None, 128, 128, 3)       0         
                                                                 
 random_translation (Random  (None, 128, 128, 3)       0         
 Translation)                                                    
                                                                 
 gaussian_noise (GaussianNo  (None, 128, 128, 3)       0         
 ise)                                                            
                                                             

In [6]:
optimizer = keras.optimizers.Adam()

# compile the model
model.compile(
    optimizer,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [7]:
EXP_ID = "10xwiderwaug"

# crate model checkpointer
saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t/"
model.save(saved_best_model_path, True, "tf")  # save all model

model_chkp = create_model_checkpointer(saved_best_model_path)
early_stopper = create_early_stopper()
lr_reducer = create_lr_reducer_on_plateau()

# Training and validating cycle.
history = model.fit(
    train_dataset,    
    epochs=EPOCHS,
    callbacks=[model_chkp, early_stopper, lr_reducer],
    verbose="auto",
)

# load best model
model.load_weights(saved_best_model_path + "weights/")

INFO:tensorflow:Assets written to: saved_models/10xwiderwaug/best_model_t/assets


INFO:tensorflow:Assets written to: saved_models/10xwiderwaug/best_model_t/assets


Epoch 1/100


2024-02-10 16:52:31.627004: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-10 16:52:32.290569: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f80305be930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-10 16:52:32.290585: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-10 16:52:32.293515: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1707594752.348868  393678 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


27/27 [==============================] - 4s 13ms/step - loss: 1.2504 - accuracy: 0.3973 - lr: 0.0010
Epoch 2/100
27/27 [==============================] - 0s 14ms/step - loss: 0.9362 - accuracy: 0.5527 - lr: 0.0010
Epoch 3/100
27/27 [==============================] - 0s 14ms/step - loss: 0.8021 - accuracy: 0.6499 - lr: 0.0010
Epoch 4/100
27/27 [==============================] - 0s 14ms/step - loss: 0.7526 - accuracy: 0.6785 - lr: 0.0010
Epoch 5/100
27/27 [==============================] - 0s 14ms/step - loss: 0.7299 - accuracy: 0.6889 - lr: 0.0010
Epoch 6/100
27/27 [==============================] - 0s 13ms/step - loss: 0.6267 - accuracy: 0.7454 - lr: 0.0010
Epoch 7/100
27/27 [==============================] - 0s 13ms/step - loss: 0.6245 - accuracy: 0.7367 - lr: 0.0010
Epoch 8/100
27/27 [==============================] - 0s 14ms/step - loss: 0.5868 - accuracy: 0.7514 - lr: 0.0010
Epoch 9/100
27/27 [==============================] - 0s 13ms/step - loss: 0.6263 - accuracy: 0.7248 - lr: 0.

In [8]:
# Evaluate the model accuracy on the validation set.
score = model.evaluate(test_dataset, verbose=1)

3/3 [==============================] - 0s 33ms/step - loss: 0.8266 - accuracy: 0.8238
